# Milestone 2

Start off by loading the paragraphs from previous milestone into a pandas dataframe. Then Set up a list of the questions.

In [1]:
import pandas as pd

df = pd.read_csv("paragraphs.csv")

questions = [
    ["What fuel is used for manufacturing of chlorine?"],
    ["What metric is used for evaluating emission?"],
    ["How can carbon emission of the processes of cement clinker be reduced?"],
    ["How is the Weighted Cogeneration Threshold calculated?"],
    ["What is carbon capture and sequestration?"],
    ["What stages does CCS consist of?"],
    ["What should be the average energy consumption of a water supply system?"],
    ["What are examples of sludge treatments?"],
    ["How is the process of anaerobic digestion?"],
    ["How is reforestation defined?"],
    ["What is the threshold of emssion for inland passenger water transport?"], 
    ["What are the requirements of reporting for electricity generation from natural gas where there might be fugative emissions?"]
]

The first method we want to test out for getting the representations for the paragraphs is Term Frequency Inverse Document Frequency (TFIDF). We will use the scikit learn package. Note that I am not filtering out the stop words here, this is can be something worth doing.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vector_corpus = vectorizer.fit_transform(df["paragraph"])

Once we have the TFIDF vectorizer and the corpus vectorized we want to iterate over the questions and get the vector. We use the linear kernal to calculate the distance between the question representation and the corpus vectors.

In [3]:
from sklearn.metrics.pairwise import linear_kernel

qcontexts = []
for question in questions:
    q_v = vectorizer.transform(question)
    lk_rank = linear_kernel(q_v, vector_corpus).flatten()
    qcontexts.append((question, df["paragraph"][lk_rank.argsort()[-1]]))

For comparison we will try to get the Doc2vec representation as well and see how it will perform. The package used here is `gensim` and it has a built in similarity measure. 

In [4]:
import gensim

def read_corpus(text, tokens_only=False):
    for i, line in enumerate(text):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

corpus = list(read_corpus(df["paragraph"].values))
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
qcontext_doc2vec = []
for question in questions:
    q1 = list(read_corpus(question, tokens_only=True))
    inferred_vector = model.infer_vector(q1[0])
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    qcontext_doc2vec.append((question, df["paragraph"][sims[0][0]]))

As a last step we print the results so we can evaluate the diffrent methods.

In [5]:
for ic,(question, context)  in enumerate(qcontexts):
    print(question[0])
    print(f"tfidf: {context}\n\ndoc2vec: {qcontext_doc2vec[ic][1]}")
    print("-"*20)

What fuel is used for manufacturing of chlorine?
tfidf: For chlorine the value corresponding to an efficient level of electricity consumption was selected as the threshold given that the main source of energy used for the production of chlorine is electricity and by improving the energy efficiency of the process as well as using low carbon electricity sources  the activity can substantially contribute to the climate change mitigation objective

doc2vec: For some corporates who manufacture as well as sell the end products they may attribute revenue to the end product rather than to the manufacturing process Technical screening criteria in this report lays out expectations of the manufacturing process When aligning testing criteria to manufacturing it may be necessary to review revenue classified to the sale of the end product by the same organisation even though the eligible activity would be assessed within the manufacturing processes This is not a supply chain assessment the corporate